# 2023-01-19 13:30
민규사원님이 주신 mobilenet v2 model 기반으로 다시 돌려보기


In [6]:
import torch
print(torch.__version__)
from torchvision.transforms import transforms
# print(torchvision.__version)



from mobilenetv2 import mobilenet_v2

1.13.1+cu117


In [7]:
emo_model = mobilenet_v2(in_channels=3, num_classes=8) 
model_path = './mobilenet_v2_affectnet_2023_01_12_07.37'

state = torch.load(model_path, map_location='cpu')
emo_model.load_state_dict(state['net'])



<All keys matched successfully>

In [5]:
img = '/home/sldev1/Desktop/Screenshot from 2023-01-19 15-29-04.png'

cap = cv2.VideoCapture(img)
ret, frame = cap.read()

print(frame.shape)


'''transform = transforms.Compose([
                        transforms.ToTensor(),])


img = transform(frame)
# img = unsqueeze(img, dim=0)
out = emo_model(img.unsqueeze(0))
print(out)
print('*********************************************************************************\n')

expr = out['expression']
print(expr)
print('*********************************************************************************\n')



expr = np.argmax(expr.cpu().detach().numpy(), axis=1)
print(expr.item())
print('*********************************************************************************\n')

# print(expr) # type = numpy'''

(694, 1279, 3)


"transform = transforms.Compose([\n                        transforms.ToTensor(),])\n\n\nimg = transform(frame)\n# img = unsqueeze(img, dim=0)\nout = emo_model(img.unsqueeze(0))\nprint(out)\nprint('*********************************************************************************\n')\n\nexpr = out['expression']\nprint(expr)\nprint('*********************************************************************************\n')\n\n\n\nexpr = np.argmax(expr.cpu().detach().numpy(), axis=1)\nprint(expr.item())\nprint('*********************************************************************************\n')\n\n# print(expr) # type = numpy"

In [15]:
img.shape

AttributeError: 'str' object has no attribute 'shape'

In [2]:
import sys


In [5]:
print(sys.version)

3.7.0 (default, Oct  9 2018, 10:31:47) 
[GCC 7.3.0]


## 수정코드

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2
# import tensorflow as tf
from emotion_torch_mobilenetV2 import Emotion
import mediapipe as mp
import csv
import glob
import re
from tqdm import tqdm

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-_rsy432w because the default path (/home/sldev1/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
dirpath = "/media/sldev1/T7/DMS2/"
# filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
filepath = glob.glob(dirpath+'*.mp4')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[3]))
filepath = filepath[7:9]
print(filepath)
emo = Emotion()
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

[]


In [3]:
for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_MobilenetV2/'+str(i)[22:-4]+'.csv', 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerow(["frame", "result"])

            while(cap.isOpened()):
                ret, frame = cap.read()
                rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detection.process(rgbframe)
                rgbframe.flags.writeable = True
                rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)

                if results.detections:
                    bbox = results.detections[0].location_data.relative_bounding_box
                    top = rgbframe.shape[0]*bbox.ymin
                    bottom = rgbframe.shape[0]*(bbox.xmin+bbox.height)
                    left = rgbframe.shape[1]*bbox.xmin
                    right = rgbframe.shape[1]*(bbox.xmin+bbox.width)

                    #pre_emo = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#                     print(np.shape(frame))q
                    frame = frame[int(top)-50:int(bottom)+50, int(left)-50:int(right)+20]
#                     frame = frame[int(top):int(bottom), int(left):int(right)]
#                     print(np.shape(frame))
                    frame = cv2.resize(frame, (256, 256))
                    frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2RGB)
                    pre_emo = emo.process(frame)
                    # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
                    cv2.putText(frame, f"Emotion : {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"Frame : {cap.get(1)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
#                     cv2.putText(frame, f"One : {one}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
#                     cv2.putText(frame, f"two : {two}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)


                    cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)

                cv2.imshow("frame", frame)

                if cv2.waitKey(1) == ord('q'):
                    break

                with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_MobilenetV2/'+str(i)[22:-4]+'.csv', 'a', newline = '') as file:
                    writer = csv.writer(file)
                    writer.writerow([cap.get(1), pre_emo])
                    file.close()
                
                if cap.get(1) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                    break
            cap.release()
            cv2.destroyAllWindows()


# 1258 frame에서 멈추는 중

0it [00:00, ?it/s]


## 01/13(금) 오류

In [ ]:
  6%|████▌                                                                      | 3/50 [2:52:17<45:43:23, 3502.20s/it]
/media/sldev1/T7/DMS2/04. 손하늘_1208.mp4
[h264 @ 0x559e98b11500] Invalid NAL unit size (7082 > 1541).
[h264 @ 0x559e98b11500] Error splitting the input into NAL units.
  6%|████▌                                                                      | 3/50 [2:54:15<45:30:00, 3485.13s/it]
---------------------------------------------------------------------------
error                                     Traceback (most recent call last)
/tmp/ipykernel_17891/1244803589.py in <module>
     12             while(cap.isOpened()):
     13                 ret, frame = cap.read()
---> 14                 rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
     15                 results = face_detection.process(rgbframe)
     16                 rgbframe.flags.writeable = True

error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
                
# Frame 2566에서 에러뜸.
# 다시 하니까 또 됨. 머노?

## 01/18 (수) csv file editing

In [1]:
pwd

'/home/sldev1/github/TIL/Internship/DMS2_SMU'

In [12]:
import glob
import re
import csv

In [13]:
dirpath = "./FER_Result_SL/"
# filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
filepath = glob.glob(dirpath+'*.csv')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[0]))
filepath = filepath[0:2]
print(filepath)


['./FER_Result_SL/01. 김형민_1201.csv', './FER_Result_SL/02. 김두용_1205.csv']


In [15]:
v = open(filepath[0])
r = csv.reader(v)
row0 = r.next()
row0.append('Valence')
print(row0)

AttributeError: '_csv.reader' object has no attribute 'next'